In [3]:
from scikeras.wrappers import KerasRegressor
import tensorflow as tf
from tensorflow import keras
import pickle
from datetime import datetime as dt
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, mean_squared_log_error, mean_absolute_percentage_error
import matplotlib.pyplot as plt
import warnings
import calendar
import collections.abc
collections.Iterable = collections.abc.Iterable
warnings.filterwarnings("ignore")

In [5]:
print(tf.config.list_physical_devices('GPU'))

[]


In [3]:
train = pd.read_csv('train.csv', index_col='date')
train.index = pd.to_datetime(train.index)
exog = pickle.load(open('exog.pkl', 'rb'))